In [1]:
from pyspark import SparkConf, SparkContext
from collections import Counter

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

def processRecord (record):
    key_word = "narodnaya"
    couple_list = []
    flag = 0
    cnt = Counter()
    for word in record:
        if flag == 1:
            couple_list.append(word.lower())
            cnt[word.lower()] += 1
        flag = 0
        if word.lower() == key_word:
            flag = 1
    return couple_list
    
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]
key_word_pair = wiki.map(processRecord)

In [2]:
key_word_reduce = key_word_pair.reduce(lambda x,y:x+y)

In [3]:
PairWordRDD = sc.parallelize(key_word_reduce)

In [4]:
PairWordCount = PairWordRDD.countByValue()

In [5]:
print len(PairWordCount)

2


In [10]:
pair_start = "narodnaya_"
for x in sorted(PairWordCount):
    print "%s%s\t%d" % (pair_start,x,PairWordCount[x])

narodnaya_gazeta	1
narodnaya_volya	9
